In [22]:
import os
from collections import deque
from dotenv import load_dotenv
from google import genai
import pandas as pd
from pydantic import BaseModel, Field
import time
from typing import Annotated, Literal

folder_path = 'data/'

ieee_file = folder_path + 'ieee_xplore.csv'
scopus_file = folder_path + 'scopus.csv'
wos_file = folder_path + 'web_of_science.xls'

In [2]:
df_ieee = pd.read_csv(ieee_file)
df_ieee

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,Early Disease Classification of Mango Leaves U...,T. N. Pham; L. V. Tran; S. V. T. Dao,School of Industrial Engineering and Managemen...,IEEE Access,26 Oct 2020,2020,8,NaN,189960,189973,...,NaN,189.0,NaN,41.0,CCBYNCND,19 Oct 2020,NaN,NaN,IEEE,IEEE Journals
1,A Multi-Modal Approach for Crop Health Mapping...,U. Shafi; R. Mumtaz; N. Iqbal; S. M. H. Zaidi;...,School of Electrical Engineering and Computer ...,IEEE Access,25 Jun 2020,2020,8,NaN,112708,112724,...,NaN,107.0,NaN,46.0,CCBY,16 Jun 2020,NaN,NaN,IEEE,IEEE Journals
2,Automatic model based dataset generation for f...,M. Di Cicco; C. Potena; G. Grisetti; A. Pretto,"Department of Computer, Control, and Managemen...",2017 IEEE/RSJ International Conference on Inte...,14 Dec 2017,2017,NaN,NaN,5188,5195,...,NaN,90.0,NaN,24.0,IEEE,14 Dec 2017,NaN,NaN,IEEE,IEEE Conferences
3,Real-Time Machine-Learning Based Crop/Weed Det...,M. Alam; M. S. Alam; M. Roman; M. Tufail; M. U...,"Department of Mechatronics Engg., University o...",2020 7th International Conference on Electrica...,28 May 2020,2020,NaN,NaN,273,280,...,NaN,89.0,NaN,17.0,IEEE,28 May 2020,NaN,NaN,IEEE,IEEE Conferences
4,Smart Agriculture Using Internet of Things,I. Mat; M. R. Mohd Kassim; A. N. Harun; I. M. ...,"MIMOS Berhad, Kuala Lumpur, MALAYSIA; MIMOS Be...",2018 IEEE Conference on Open Systems (ICOS),3 Feb 2019,2018,NaN,NaN,54,59,...,NaN,76.0,NaN,11.0,IEEE,3 Feb 2019,NaN,NaN,IEEE,IEEE Conferences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,Using AI and IoT at the Edge of the network,S. Lakrouni; M. Sebgui; S. Bah,"Smart Communication Research Team, Mohamedia S...",2022 8th International Conference on Optimizat...,8 Nov 2022,2022,NaN,NaN,1,6,...,NaN,NaN,NaN,17.0,IEEE,8 Nov 2022,NaN,NaN,IEEE,IEEE Conferences
309,Using Deep Learning for Precise Plant Patholog...,P. Kaushik; K. S. Gill; N. Thapliyal; R. S. Rawat,Chitkara University Institute of Engineering a...,2024 IEEE International Conference on Smart Po...,25 Nov 2024,2024,NaN,NaN,1,4,...,NaN,NaN,NaN,20.0,IEEE,25 Nov 2024,NaN,NaN,IEEE,IEEE Conferences
310,VisionaryX-9: Efficient plant disease predicti...,I. S. Rajput; K. Bhardwaj; D. Sanwal,"CSE Department, Grahpic Era Hill University, H...",2024 IEEE International Conference on Communic...,29 Nov 2024,2024,NaN,NaN,1,6,...,NaN,NaN,NaN,17.0,IEEE,29 Nov 2024,NaN,NaN,IEEE,IEEE Conferences
311,Wheat Leaf Disease Detection Using CapsNet in ...,N. Subash; S. Sushanth; T. S. Shivani; P. Sooraj,"Department of ECE VNR VJIET, Hyderabad, India;...",2024 2nd International Conference on Cyber Phy...,26 Mar 2025,2024,NaN,NaN,1,6,...,NaN,NaN,NaN,23.0,IEEE,26 Mar 2025,NaN,NaN,IEEE,IEEE Conferences


In [3]:
df_ieee = df_ieee[['DOI', 'Document Title', 'Authors', 'Publication Year', 'Abstract', 'Article Citation Count']]

df_ieee = df_ieee.rename(columns={
    'Document Title': 'Title',
    'Publication Year': 'Year',
    'Article Citation Count': 'Citations'
})

df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2020.3031914,Early Disease Classification of Mango Leaves U...,T. N. Pham; L. V. Tran; S. V. T. Dao,2020,"Plant disease, especially crop plants, is a ma...",189.0
1,10.1109/ACCESS.2020.3002948,A Multi-Modal Approach for Crop Health Mapping...,U. Shafi; R. Mumtaz; N. Iqbal; S. M. H. Zaidi;...,2020,The agriculture sector holds paramount importa...,107.0
2,10.1109/IROS.2017.8206408,Automatic model based dataset generation for f...,M. Di Cicco; C. Potena; G. Grisetti; A. Pretto,2017,Selective weeding is one of the key challenges...,90.0
3,10.1109/ICEEE49618.2020.9102505,Real-Time Machine-Learning Based Crop/Weed Det...,M. Alam; M. S. Alam; M. Roman; M. Tufail; M. U...,2020,Traditional agrochemical spraying techniques o...,89.0
4,10.1109/ICOS.2018.8632817,Smart Agriculture Using Internet of Things,I. Mat; M. R. Mohd Kassim; A. N. Harun; I. M. ...,2018,Recent researches hypothetically shown the pot...,76.0
...,...,...,...,...,...,...
308,10.1109/ICOA55659.2022.9934603,Using AI and IoT at the Edge of the network,S. Lakrouni; M. Sebgui; S. Bah,2022,"In recent years, IoT devices have been widely ...",NaN
309,10.1109/ICSPCRE62303.2024.10675064,Using Deep Learning for Precise Plant Patholog...,P. Kaushik; K. S. Gill; N. Thapliyal; R. S. Rawat,2024,Plant diseases are posing an increasing danger...,NaN
310,10.1109/IICCCS61609.2024.10763666,VisionaryX-9: Efficient plant disease predicti...,I. S. Rajput; K. Bhardwaj; D. Sanwal,2024,"Agriculture, which is the foundation of worldw...",NaN
311,10.1109/ICPEEV63032.2024.10931975,Wheat Leaf Disease Detection Using CapsNet in ...,N. Subash; S. Sushanth; T. S. Shivani; P. Sooraj,2024,Various diseases that affect the growth of whe...,NaN


In [4]:
df_ieee = df_ieee.dropna(subset=['DOI'])
df_ieee = df_ieee.dropna(subset=['Citations']).loc[df_ieee['Citations'] > 1]
df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2020.3031914,Early Disease Classification of Mango Leaves U...,T. N. Pham; L. V. Tran; S. V. T. Dao,2020,"Plant disease, especially crop plants, is a ma...",189.0
1,10.1109/ACCESS.2020.3002948,A Multi-Modal Approach for Crop Health Mapping...,U. Shafi; R. Mumtaz; N. Iqbal; S. M. H. Zaidi;...,2020,The agriculture sector holds paramount importa...,107.0
2,10.1109/IROS.2017.8206408,Automatic model based dataset generation for f...,M. Di Cicco; C. Potena; G. Grisetti; A. Pretto,2017,Selective weeding is one of the key challenges...,90.0
3,10.1109/ICEEE49618.2020.9102505,Real-Time Machine-Learning Based Crop/Weed Det...,M. Alam; M. S. Alam; M. Roman; M. Tufail; M. U...,2020,Traditional agrochemical spraying techniques o...,89.0
4,10.1109/ICOS.2018.8632817,Smart Agriculture Using Internet of Things,I. Mat; M. R. Mohd Kassim; A. N. Harun; I. M. ...,2018,Recent researches hypothetically shown the pot...,76.0
...,...,...,...,...,...,...
153,10.1109/BigData47090.2019.9006264,Optimization of arable land use towards meat-f...,V. Kuzmanovski; D. E. Larsen; C. B. Henriksen,2019,UN Sustainable Development Goals and the Paris...,2.0
154,10.1109/CNS.2017.8228620,SPRIDE: Scalable and private continual geo-dis...,Q. Yan; H. Yang; M. C. Vuran; S. Irmak,2017,Precision agriculture relies on real-time data...,2.0
155,10.1109/CoNTESA50436.2020.9302856,Smart Villages – Geospatial Technology in Alba...,G. Imeraj; B. Çiço,2020,In the same way that we understand what is the...,2.0
156,10.1109/ICIRCA54612.2022.9985668,Study on Machine Learning Techniques used for ...,M. C. M; B. Prince; P. N,2022,This paper presents the various crop yield pre...,2.0


# Scopus

In [5]:
df_scopus = pd.read_csv(scopus_file)
df_scopus

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"M.U., Rehman, Mobeen Ur; H., Eesaar, Hassan; Z...","Rehman, Mobeen Ur (57214040677); Eesaar, Hassa...",57214040677; 58753584800; 57204003769; 7006006...,Advanced drone-based weed detection using feat...,2024,Knowledge-Based Systems,305,NaN,112655,NaN,...,09507051,NaN,KNSYE,NaN,English,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85207767973
1,"S., Ahmed, Salman; S.N.K., Marwat, Safdar Nawa...","Ahmed, Salman (57223755209); Marwat, Safdar Na...",57223755209; 55235238800; 6506167977; 57536804...,IoT based intelligent pest management system f...,2024,Scientific Reports,14,1,31917,NaN,...,20452322,NaN,NaN,39738391.0,English,Article,Final,All Open Access; Gold Open Access; Green Final...,Scopus,2-s2.0-85213694186
2,"L., Liu, Lei; F., Yang, Fan; X., Liu, Xiangyi;...","Liu, Lei (57211059123); Yang, Fan (57219436983...",57211059123; 57219436983; 59410346500; 5487972...,A review of the current status and common key ...,2024,Computers and Electronics in Agriculture,227,NaN,109630,NaN,...,01681699,NaN,CEAGE,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85209123968
3,"K.R., Nirmal, Khyati R.; M.R., Chilukala, Mahe...","Nirmal, Khyati R. (37011632500); Chilukala, Ma...",37011632500; 57215910893; 57197533178; 5812195...,Advancements in Environmental Data Analysis fo...,2024,Remote Sensing in Earth Systems Sciences,7,4,NaN,670.0,...,25208195; 25208209,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85207019307
4,"Z., Li, Zhuolin; D., Wang, Dashuai; Q., Yan, Q...","Li, Zhuolin (57422151200); Wang, Dashuai (5719...",57422151200; 57193409256; 59351037400; 5935108...,Winter wheat weed detection based on deep lear...,2024,Computers and Electronics in Agriculture,227,NaN,109448,NaN,...,01681699,NaN,CEAGE,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85205426157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,"D.B., Lobell, David B.; G., Azzari, George","Lobell, David B. (6603095001); Azzari, George ...",6603095001; 37046713400,Satellite detection of rising maize yield hete...,2017,Environmental Research Letters,12,1,014014,NaN,...,17489326,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85011396320
163,"F., Edwards-Murphy, Fiona; M., Magno, Michele;...","Edwards-Murphy, Fiona (56769022900); Magno, Mi...",56769022900; 23097444700; 53881992200; 7006893...,B+WSN: Smart beehive with preliminary decision...,2016,Computers and Electronics in Agriculture,124,NaN,NaN,211.0,...,01681699,NaN,CEAGE,NaN,English,Article,Final,NaN,Scopus,2-s2.0-84963830865
164,"C., Mongrédien, Cécile; J.P., Doyen, Jean Phil...","Mongrédien, Cécile (16230537600); Doyen, Jean ...",16230537600; 57193874598; 55605394700; 3681549...,Centimeter-level positioning for UAVs and othe...,2016,NaN,2,NaN,NaN,1441.0,...,NaN,9781510834101; 9781510817258,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85017307104
165,"R., Stanaway, Richard; C., Roberts, Craig","Stanaway, Richard (6505939390); Roberts, Craig...",6505939390; 7404402516,A high-precision deformation model to support ...,2016,International Association of Geodesy Symposia,NaN,NaN,NaN,149.0,...,21979359; 09399585,9783319456287; 9783319108360; 9783030542665; 9...,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-84984910832


In [6]:
df_scopus = df_scopus[['DOI', 'Title', 'Authors', 'Year', 'Abstract', 'Cited by']]

df_scopus = df_scopus.rename(columns={
    'Cited by': 'Citations'
})

df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.knosys.2024.112655,Advanced drone-based weed detection using feat...,"M.U., Rehman, Mobeen Ur; H., Eesaar, Hassan; Z...",2024,This research addresses the pressing challenge...,32
1,10.1038/s41598-024-83012-3,IoT based intelligent pest management system f...,"S., Ahmed, Salman; S.N.K., Marwat, Safdar Nawa...",2024,Despite seemingly inexorable imminent risks of...,7
2,10.1016/j.compag.2024.109630,A review of the current status and common key ...,"L., Liu, Lei; F., Yang, Fan; X., Liu, Xiangyi;...",2024,The arrival of the Industrial Revolution 4.0 a...,9
3,10.1007/s41976-024-00154-8,Advancements in Environmental Data Analysis fo...,"K.R., Nirmal, Khyati R.; M.R., Chilukala, Mahe...",2024,Weather significantly influences agricultural ...,0
4,10.1016/j.compag.2024.109448,Winter wheat weed detection based on deep lear...,"Z., Li, Zhuolin; D., Wang, Dashuai; Q., Yan, Q...",2024,Weeds seriously threaten winter wheat yields. ...,10
...,...,...,...,...,...,...
162,10.1088/1748-9326/aa5371,Satellite detection of rising maize yield hete...,"D.B., Lobell, David B.; G., Azzari, George",2017,The future trajectory of crop yields in the Un...,53
163,10.1016/j.compag.2016.04.008,B+WSN: Smart beehive with preliminary decision...,"F., Edwards-Murphy, Fiona; M., Magno, Michele;...",2016,United Nations reports throughout recent years...,114
164,NaN,Centimeter-level positioning for UAVs and othe...,"C., Mongrédien, Cécile; J.P., Doyen, Jean Phil...",2016,"To date, high costs and complexity have confin...",21
165,10.1007/1345_2015_31,A high-precision deformation model to support ...,"R., Stanaway, Richard; C., Roberts, Craig",2016,This paper describes a gridded kinematic repre...,0


In [7]:
df_scopus = df_scopus.dropna(subset=['DOI'])
df_scopus = df_scopus.loc[df_scopus['Citations'] > 1]
df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.knosys.2024.112655,Advanced drone-based weed detection using feat...,"M.U., Rehman, Mobeen Ur; H., Eesaar, Hassan; Z...",2024,This research addresses the pressing challenge...,32
1,10.1038/s41598-024-83012-3,IoT based intelligent pest management system f...,"S., Ahmed, Salman; S.N.K., Marwat, Safdar Nawa...",2024,Despite seemingly inexorable imminent risks of...,7
2,10.1016/j.compag.2024.109630,A review of the current status and common key ...,"L., Liu, Lei; F., Yang, Fan; X., Liu, Xiangyi;...",2024,The arrival of the Industrial Revolution 4.0 a...,9
4,10.1016/j.compag.2024.109448,Winter wheat weed detection based on deep lear...,"Z., Li, Zhuolin; D., Wang, Dashuai; Q., Yan, Q...",2024,Weeds seriously threaten winter wheat yields. ...,10
5,10.1016/j.future.2024.07.026,SHIELD: A Secure Heuristic Integrated Environm...,"A.K., Kaushal, Ashish Kumar; O., Almurshed, Os...",2024,The increasing adoption of edge computing in r...,2
...,...,...,...,...,...,...
160,10.1017/wet.2017.118,Geographic Information System for Pigweed Dist...,"R.S., Fletcher, Reginald S.; K.N., Reddy, Kris...",2018,"In the southeastern United States, Amaranthus,...",2
161,10.1117/12.2262804,Crop classification using temporal stacks of m...,"D.I., Moody, Daniela I.; S.P., Brumby, Steven ...",2017,"The increase in performance, availability, and...",6
162,10.1088/1748-9326/aa5371,Satellite detection of rising maize yield hete...,"D.B., Lobell, David B.; G., Azzari, George",2017,The future trajectory of crop yields in the Un...,53
163,10.1016/j.compag.2016.04.008,B+WSN: Smart beehive with preliminary decision...,"F., Edwards-Murphy, Fiona; M., Magno, Michele;...",2016,United Nations reports throughout recent years...,114


# Web of Science

In [8]:
df_wos = pd.read_excel(wos_file)
df_wos

,Publication Type,Authors,Book Authors,Group Authors,Book Group Authors,Researcher Ids,ORCIDs,Book Editors,Author - Arabic,Grant Principal Investigator,...,Copyright,Degree Name,Institution Address,Institution,Dissertation and Thesis Subjects,Author Keywords,Indexed Date,UT (Unique ID),Pubmed Id,Unnamed: 78
0,J,"Bertoglio, Riccardo; Corbo, Chiara; Renga, Fil...",NaN,NaN,NaN,"; corbo, Chiara/GYE-0076-2022","Bertoglio, Riccardo/0000-0001-6796-9463;",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-21,WOS:000704814900001,NaN,NaN
1,J,"Kim, Semo; Bae, Seoung-Hun; Kim, Min-Kwan; Kan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-08,WOS:001017551500002,NaN,NaN
2,J,"Jin, Huidong; Bakar, K. Shuvo; Henderson, Bren...",NaN,NaN,NaN,"Gobbett, David/F-8910-2010; Bramley, Rob/A-551...","Bramley, Rob/0000-0003-0643-7409; Bakar, K Shu...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-20,WOS:000640359200008,NaN,NaN
3,J,"Assimakopoulos, Fotis; Vassilakis, Costas; Mar...",NaN,NaN,NaN,"Kotis, Konstantinos/B-1883-2009; Costas, Vassi...","Kotis, Konstantinos/0000-0001-7838-9691; Marga...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-03,WOS:001364728900001,NaN,NaN
4,J,"Musanase, Christine; Vodacek, Anthony; Hanyurw...",NaN,NaN,NaN,"Hanyurwimfura, Damien/U-5578-2017; Vodacek, An...","Hanyurwimfura, Damien/0000-0002-6290-1256; MUS...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-17,WOS:001107851200001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,J,"Martinez-Baron, Deissy; de Anton, Marina Alarc...",NaN,NaN,NaN,NaN,"Alarcon de Anton, Marina/0009-0005-6447-0460",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-01,WOS:001276432100001,NaN,NaN
150,J,"Matthews, Lisa; Strauss, Johann A.; Reinsch, T...",NaN,NaN,NaN,"Kluß, Christof/LMN-2728-2024; Swanepoel, Piete...","Taube, Friedhelm/0000-0001-7175-2881",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-08,WOS:001434291000001,NaN,NaN
151,J,"Moysiadis, Vasileios; Sarigiannidis, Panagioti...",NaN,NaN,NaN,"; Sarigiannidis, Panagiotis/O-5246-2017","Vitsas, Vasileios/0000-0001-7055-6117; Khelifi...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-21,WOS:000621106000006,NaN,NaN
152,J,"Nejad, Seyed Mahdi Mirhoseini; Abbasi-Moghadam...",NaN,NaN,NaN,"; Sharifi, Alireza/H-8497-2016; Abbasi-Moghada...","Mirhoseini Nejad, Seyed Mahdi/0000-0002-0454-8...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-27,WOS:001336298000014,NaN,NaN


In [9]:
df_wos = df_wos[['DOI', 'Article Title', 'Authors', 'Publication Year', 'Abstract', 'Times Cited, All Databases']]

df_wos = df_wos.rename(columns={
    'Article Title': 'Title',
    'Publication Year': 'Year',
    'Times Cited, All Databases': 'Citations'
})

df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2021.3115258,The Digital Agricultural Revolution: A Bibliom...,"Bertoglio, Riccardo; Corbo, Chiara; Renga, Fil...",2021,The application of digital technologies in agr...,70
1,10.1007/s42405-023-00617-0,Deep Learning Performance Comparison Using Mul...,"Kim, Semo; Bae, Seoung-Hun; Kim, Min-Kwan; Kan...",2023,This study aims to develop an efficient farmla...,1
2,10.1016/j.biosystemseng.2021.02.009,An efficient geostatistical analysis tool for ...,"Jin, Huidong; Bakar, K. Shuvo; Henderson, Bren...",2021,On farm experimentation (OFE) has been a long-...,9
3,10.3390/electronics13224362,Artificial Intelligence Tools for the Agricult...,"Assimakopoulos, Fotis; Vassilakis, Costas; Mar...",2024,This article explores the transformative poten...,17
4,10.3390/agriculture13112141,Data-Driven Analysis and Machine Learning-Base...,"Musanase, Christine; Vodacek, Anthony; Hanyurw...",2023,Agriculture plays a key role in global food se...,34
...,...,...,...,...,...,...
149,10.3389/fsufs.2024.1363101,Climate-smart agriculture reduces capital-base...,"Martinez-Baron, Deissy; de Anton, Marina Alarc...",2024,Introduction Climate change poses a significan...,3
150,10.1016/j.agsy.2024.104218,Legumes and livestock in no-till crop rotation...,"Matthews, Lisa; Strauss, Johann A.; Reinsch, T...",2025,Context: Crop rotation is seen as a 'Climate-S...,2
151,10.1016/j.cosrev.2020.100345,Smart Farming in Europe,"Moysiadis, Vasileios; Sarigiannidis, Panagioti...",2021,Smart Farming is the new term in the agricultu...,142
152,10.1109/JSTARS.2024.3464411,ConvLSTM-ViT: A Deep Neural Network for Crop Y...,"Nejad, Seyed Mahdi Mirhoseini; Abbasi-Moghadam...",2024,This article introduces an approach for soybea...,33


In [10]:
df_wos = df_wos.dropna(subset=['DOI', 'Abstract'])
df_wos = df_wos.loc[df_wos['Year'] != 2025]
df_wos = df_wos.loc[df_wos['Citations'] > 1]
df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2021.3115258,The Digital Agricultural Revolution: A Bibliom...,"Bertoglio, Riccardo; Corbo, Chiara; Renga, Fil...",2021,The application of digital technologies in agr...,70
2,10.1016/j.biosystemseng.2021.02.009,An efficient geostatistical analysis tool for ...,"Jin, Huidong; Bakar, K. Shuvo; Henderson, Bren...",2021,On farm experimentation (OFE) has been a long-...,9
3,10.3390/electronics13224362,Artificial Intelligence Tools for the Agricult...,"Assimakopoulos, Fotis; Vassilakis, Costas; Mar...",2024,This article explores the transformative poten...,17
4,10.3390/agriculture13112141,Data-Driven Analysis and Machine Learning-Base...,"Musanase, Christine; Vodacek, Anthony; Hanyurw...",2023,Agriculture plays a key role in global food se...,34
6,10.3390/rs14030559,A Review of Deep Learning in Multiscale Agricu...,"Wang, Dashuai; Cao, Wujing; Zhang, Fan; Li, Zh...",2022,"Population growth, climate change, and the wor...",145
...,...,...,...,...,...,...
147,10.1016/j.rineng.2024.102283,Enhancing water management in smart agricultur...,"Et-taibi, Bouali; Abid, Mohamed Riduan; Boufou...",2024,It is widely acknowledged that traditional agr...,46
148,10.1007/s10586-021-03397-y,Extreme learning machine for plant diseases cl...,"Aqel, Darah; Al-Zubi, Shadi; Mughaid, Ala; Jar...",2022,"Nowadays, the economy of countries highly depe...",44
149,10.3389/fsufs.2024.1363101,Climate-smart agriculture reduces capital-base...,"Martinez-Baron, Deissy; de Anton, Marina Alarc...",2024,Introduction Climate change poses a significan...,3
151,10.1016/j.cosrev.2020.100345,Smart Farming in Europe,"Moysiadis, Vasileios; Sarigiannidis, Panagioti...",2021,Smart Farming is the new term in the agricultu...,142


# Final DataFrame

In [11]:
df_all = pd.concat([df_ieee, df_scopus, df_wos], ignore_index=True)
df_all

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2020.3031914,Early Disease Classification of Mango Leaves U...,T. N. Pham; L. V. Tran; S. V. T. Dao,2020,"Plant disease, especially crop plants, is a ma...",189.0
1,10.1109/ACCESS.2020.3002948,A Multi-Modal Approach for Crop Health Mapping...,U. Shafi; R. Mumtaz; N. Iqbal; S. M. H. Zaidi;...,2020,The agriculture sector holds paramount importa...,107.0
2,10.1109/IROS.2017.8206408,Automatic model based dataset generation for f...,M. Di Cicco; C. Potena; G. Grisetti; A. Pretto,2017,Selective weeding is one of the key challenges...,90.0
3,10.1109/ICEEE49618.2020.9102505,Real-Time Machine-Learning Based Crop/Weed Det...,M. Alam; M. S. Alam; M. Roman; M. Tufail; M. U...,2020,Traditional agrochemical spraying techniques o...,89.0
4,10.1109/ICOS.2018.8632817,Smart Agriculture Using Internet of Things,I. Mat; M. R. Mohd Kassim; A. N. Harun; I. M. ...,2018,Recent researches hypothetically shown the pot...,76.0
...,...,...,...,...,...,...
371,10.1016/j.rineng.2024.102283,Enhancing water management in smart agricultur...,"Et-taibi, Bouali; Abid, Mohamed Riduan; Boufou...",2024,It is widely acknowledged that traditional agr...,46.0
372,10.1007/s10586-021-03397-y,Extreme learning machine for plant diseases cl...,"Aqel, Darah; Al-Zubi, Shadi; Mughaid, Ala; Jar...",2022,"Nowadays, the economy of countries highly depe...",44.0
373,10.3389/fsufs.2024.1363101,Climate-smart agriculture reduces capital-base...,"Martinez-Baron, Deissy; de Anton, Marina Alarc...",2024,Introduction Climate change poses a significan...,3.0
374,10.1016/j.cosrev.2020.100345,Smart Farming in Europe,"Moysiadis, Vasileios; Sarigiannidis, Panagioti...",2021,Smart Farming is the new term in the agricultu...,142.0


In [12]:
df_all = df_all.drop_duplicates(subset=['DOI'])
df_all = df_all.drop(columns=['Citations'])
df_all

,DOI,Title,Authors,Year,Abstract
0,10.1109/ACCESS.2020.3031914,Early Disease Classification of Mango Leaves U...,T. N. Pham; L. V. Tran; S. V. T. Dao,2020,"Plant disease, especially crop plants, is a ma..."
1,10.1109/ACCESS.2020.3002948,A Multi-Modal Approach for Crop Health Mapping...,U. Shafi; R. Mumtaz; N. Iqbal; S. M. H. Zaidi;...,2020,The agriculture sector holds paramount importa...
2,10.1109/IROS.2017.8206408,Automatic model based dataset generation for f...,M. Di Cicco; C. Potena; G. Grisetti; A. Pretto,2017,Selective weeding is one of the key challenges...
3,10.1109/ICEEE49618.2020.9102505,Real-Time Machine-Learning Based Crop/Weed Det...,M. Alam; M. S. Alam; M. Roman; M. Tufail; M. U...,2020,Traditional agrochemical spraying techniques o...
4,10.1109/ICOS.2018.8632817,Smart Agriculture Using Internet of Things,I. Mat; M. R. Mohd Kassim; A. N. Harun; I. M. ...,2018,Recent researches hypothetically shown the pot...
...,...,...,...,...,...
370,10.1016/j.heliyon.2024.e29564,Exploring the application of ICTs in decarboni...,"El Jaouhari, Asmae; Arif, Jabir; Samadhiya, As...",2024,The contemporary agricultural supply chain nec...
371,10.1016/j.rineng.2024.102283,Enhancing water management in smart agricultur...,"Et-taibi, Bouali; Abid, Mohamed Riduan; Boufou...",2024,It is widely acknowledged that traditional agr...
372,10.1007/s10586-021-03397-y,Extreme learning machine for plant diseases cl...,"Aqel, Darah; Al-Zubi, Shadi; Mughaid, Ala; Jar...",2022,"Nowadays, the economy of countries highly depe..."
373,10.3389/fsufs.2024.1363101,Climate-smart agriculture reduces capital-base...,"Martinez-Baron, Deissy; de Anton, Marina Alarc...",2024,Introduction Climate change poses a significan...


# Abstracts Screening with AI

In [15]:
load_dotenv()
MODEL = os.getenv("MODEL")
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words"
)
print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


AI learns patterns from data to make predictions or decisions.


In [35]:
class ReviewDecision(BaseModel):
    decision: Annotated[Literal["yes", "no", "maybe"], Field(description="Decision about including the paper:"
                                                                         "- yes: there is clear evidence that the paper uses, shares, or discusses datasets according to ALL inclusion and exclusion criteria."
                                                                         "- no: there is clear evidence that the paper should be excluded (it does not meet ALL inclusion criteria or meets AT LEAST one exclusion criteria)."
                                                                         "- maybe: there is no clear evidence either way; checking the full text is recommended."
                                                             )]
    reason: Annotated[str, Field(description="Short explanation for the decision")]
    data_source: Annotated[str, Field(
        description="Official name of the data source used in the paper if decision='yes' and the dataset is explicitly mentioned."
                    "In ALL of other cases leave as an empty string.")]


# Rate limiter: max 15 requests every 60 seconds, until the window allows more
_RATE_WINDOW_SEC = 60.0
_RATE_MAX_CALLS = 15
_rate_calls_log = deque()
_sent_calls_count = 0


def _rate_limit_block_until_allowed():
    while True:
        now = time.monotonic()
        # purge timestamps outside the window
        while _rate_calls_log and (now - _rate_calls_log[0]) >= _RATE_WINDOW_SEC:
            _rate_calls_log.popleft()
        if len(_rate_calls_log) < _RATE_MAX_CALLS:
            _rate_calls_log.append(now)
            return


def is_open_data_relevant(title: str, abstract: str, client, model="gemini-2.5-flash-lite", max_retries=3):
    """
    Evaluate if a paper indicates the use, sharing, or discussion of data sources in Agriculture.

    Returns:
    ReviewDecision: structured output containing:
      - decision: 'yes', 'no', or 'maybe'
      - reason: short explanation
      - data_source: official name of dataset if decision='yes' and explicitly mentioned, else empty string
    """
    if not title or str(title).strip() == "" or not abstract or str(abstract).strip() == "":
        return ReviewDecision(decision="no", reason="Title or abstract is missing", data_source="")

    prompt = (
        "Based on the following Inclusion and Exclusion Criteria, decide if the paper should be included.\n\n"
        "Inclusion Criteria:\n"
        "(a) The abstract mentions the use or production of datasets, databases, or any other type of data sources.\n"
        "(b) The data sources used is related to agriculture or a close-related field (climate, crops, farms, plants, soil, satellite).\n"
        "(c) The data sources used have global coverage, they shouldn't be related to regional/local data.\n\n"
        "Exclusion Criteria:\n"
        "(a) The abstract mentions that the data is not available or must be requested from authors.\n"
        "(b) The data used or generated is not free, there is a cost associated.\n\n"
        "The main goal is to check if the title and abstract provide enough information to consider that the paper is using, sharing, or discussing data sources in Agriculture.\n"
        f"Title:\n{title}\n\nAbstract:\n{abstract}"
    )

    global _sent_calls_count
    backoff = 2

    for attempt in range(max_retries):
        # Block (without sleep) until the rate limiter allows the call
        _rate_limit_block_until_allowed()
        try:
            _sent_calls_count += 1
            if _sent_calls_count % 10 == 0:
                print(f"[GEMINI] Completed {_sent_calls_count} total requests.")
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": ReviewDecision
                },
            )
            return resp.parsed
        except Exception as e:
            print(f"[attempt {attempt + 1}] Gemini error: {e}")
            time.sleep(backoff)
            backoff *= 2
            continue
    return ReviewDecision(decision="maybe", reason="All retries failed", data_source="")


test_df = df_all.copy()
results = test_df.apply(
    lambda row: is_open_data_relevant(row['Title'], row['Abstract'], client), axis=1
)

test_df['Include'] = results.apply(lambda x: x.decision)
test_df['Reason'] = results.apply(lambda x: x.reason)
test_df['Data Source'] = results.apply(lambda x: x.data_source)

test_df[['DOI', 'Title', 'Include', 'Reason', 'Data Source']]

[GEMINI] Completed 10 total requests.
[GEMINI] Completed 20 total requests.
[GEMINI] Completed 30 total requests.
[GEMINI] Completed 40 total requests.
[GEMINI] Completed 50 total requests.
[GEMINI] Completed 60 total requests.
[GEMINI] Completed 70 total requests.
[GEMINI] Completed 80 total requests.
[GEMINI] Completed 90 total requests.
[GEMINI] Completed 100 total requests.
[GEMINI] Completed 110 total requests.
[GEMINI] Completed 120 total requests.
[GEMINI] Completed 130 total requests.
[GEMINI] Completed 140 total requests.
[GEMINI] Completed 150 total requests.
[GEMINI] Completed 160 total requests.
[GEMINI] Completed 170 total requests.
[GEMINI] Completed 180 total requests.
[GEMINI] Completed 190 total requests.
[GEMINI] Completed 200 total requests.
[GEMINI] Completed 210 total requests.
[GEMINI] Completed 220 total requests.
[GEMINI] Completed 230 total requests.
[GEMINI] Completed 240 total requests.
[GEMINI] Completed 250 total requests.
[GEMINI] Completed 260 total reque

,DOI,Title,Include,Reason,Data Source
0,10.1109/ACCESS.2020.3031914,Early Disease Classification of Mango Leaves U...,maybe,The abstract mentions the creation of a datase...,
1,10.1109/ACCESS.2020.3002948,A Multi-Modal Approach for Crop Health Mapping...,no,While the abstract mentions using remote sensi...,
2,10.1109/IROS.2017.8206408,Automatic model based dataset generation for f...,yes,The abstract clearly states the generation of ...,synthetic agricultural dataset
3,10.1109/ICEEE49618.2020.9102505,Real-Time Machine-Learning Based Crop/Weed Det...,maybe,The abstract mentions the creation and use of ...,
4,10.1109/ICOS.2018.8632817,Smart Agriculture Using Internet of Things,maybe,The abstract mentions the use of IoT in agricu...,
...,...,...,...,...,...
370,10.1016/j.heliyon.2024.e29564,Exploring the application of ICTs in decarboni...,maybe,The abstract mentions the use of 57 research p...,
371,10.1016/j.rineng.2024.102283,Enhancing water management in smart agricultur...,maybe,The abstract mentions the use of big data acqu...,
372,10.1007/s10586-021-03397-y,Extreme learning machine for plant diseases cl...,no,The abstract mentions a dataset of 73 plant le...,
373,10.3389/fsufs.2024.1363101,Climate-smart agriculture reduces capital-base...,no,The abstract mentions a dataset of 267 househo...,


In [37]:
unique_sources = test_df['Data Source'].unique().tolist()
unique_sources

['',
 'synthetic agricultural dataset',
 'open dataset',
 'Web of Science',
 'Interplanetary File System (IPFS)',
 'hyperspectral UAV dataset',
 'cotton boll dataset, wheat ear dataset',
 'pomegranate dataset',
 'ADCON RTUs, Grafana, influxDB database',
 'PlantVillage dataset, cassava dataset, Tomato leaves dataset',
 'AIWeeds',
 'lettuce point cloud dataset',
 'database of plant leaf images',
 'Red Clover Ground Cover (RCGC) dataset',
 'neural-inertial navigation dataset for precision agricultural robots',
 'precision agriculture information technologies',
 'dataset of citrus leaf images',
 'MinneApple Apple segmentation dataset',
 'phenotyping dataset',
 'Global Wheat Head Detection, Large Wheat Disease Classification Dataset',
 'NDVI',
 'Insect-1M',
 'tomato leaves dataset',
 'mustard plant dataset',
 'smart farming network datasets',
 'PlantPathology',
 'Plant Village Dataset',
 'SMAP',
 'tomato maturity dataset',
 'Scopus database',
 'CCAFS open dataset',
 'OSF drone image dataset

In [38]:
n_unique = test_df['Data Source'].nunique()
print(f"Numero di valori unici in Data Source: {n_unique}")

Numero di valori unici in Data Source: 84
